In [ ]:
from framework.dataprovider import *
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Reshape
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.optimizers import SGD
import numpy as np

rng = np.random.RandomState(123456)

timehorizons = [1, 2, 4, 10, 25, 50, 100]
#timehorizons = [1, 2, 4]
shapes = []
scores = []
testRatios = []
numEpochs = 5

for timehorizon in timehorizons:
    print 'testing %s' % timehorizon
    rng.seed(123456)
    dp = TimeWindow3DDataProvider(timehorizon, rng=rng)
    data, targets = dp.getTrainingData()
    shapes.append(data.shape)
    print data.shape[1:]
    model = Sequential()
    model.add(Convolution2D(timehorizon, 2, 2, input_shape=data.shape[1:], init="he_normal"))
    model.add(Activation("tanh"))
    model.add(Convolution2D(timehorizon * 2, 3, 3, init="he_normal"))
    model.add(Activation("tanh"))
    model.add(Reshape([-1]))
    model.add(Dense(output_dim=300, init="he_normal"))
    model.add(Activation("tanh"))
    model.add(Dense(output_dim=2, init="he_normal"))
    model.add(Activation("softmax"))
    
    #sgd = SGD(lr=0.1, decay=1e-2, momentum=0.5, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    print 'training model'
    model.fit(data, targets, show_accuracy=True, nb_epoch=numEpochs)
    
    testData, testTargets = dp.getTestData()
    testRatios.append(float(np.sum(testTargets[:,0]))/testTargets.shape[0])
    predictions = model.predict(testData)
    ac = np.argmax(predictions, axis=-1)
    score = np.sum(testTargets[xrange(ac.shape[0]), ac])/float(testTargets.shape[0])
    #score = model.evaluate(testData, testTargets, show_accuracy=1, verbose=1)
    scores.append(score)
    
print "%-15s | %-15s | %-10s | %-5s" % ('time horizon', 'shape', 'test ratio', 'score')
print '--------------------------------------------'
for i in xrange(len(timehorizons)):
    print "%-15s | %-15s | %-10s | %-5s" % (timehorizons[i], shapes[i], testRatios[i], scores[i])



testing 1
(1, 388, 4)
training model
Epoch 1/5
3534/3534 [==============================] - 0s - loss: 1.1153 - acc: 0.4997     
Epoch 2/5
3534/3534 [==============================] - 0s - loss: 0.7288 - acc: 0.5003     
Epoch 3/5
3534/3534 [==============================] - 0s - loss: 0.6988 - acc: 0.5161     
Epoch 4/5
3534/3534 [==============================] - 0s - loss: 0.6926 - acc: 0.5161     
Epoch 5/5
3534/3534 [==============================] - 0s - loss: 0.6924 - acc: 0.5192     
testing 2
(2, 388, 4)
training model
Epoch 1/5
2304/3533 [==================>...........] - ETA: 0s - loss: 8.0342 - acc: 0.4809

In [11]:
print ac[:5]
print zip(xrange(5), ac[:5])
print testTargets[xrange(5), ac[:5]]
print enumerate(ac[:5])

[0 0 0 0 0]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0)]
[ 1.  0.  1.  1.  0.]
